In [4]:
from sia.satellite import sentinel2
from sia.utils.helper import *
from sia.utils.raster import *
from sia.indices.indice import *
import os
import errno
from collections import defaultdict
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
s2 = sentinel2.Sentinel2()

# # s2_tile = '/home/ec2-user/SageMaker/sumit/crop-classification/unsupcc/satellite/satellite_tiles/s2_tile.shp'
# shape_file = 'data/Farmdata_Jaleubari_Ladhasar_Ratangarh_Churu_1-polygon.shp'
shape_file = 'data/india_shp/districts/farrukhabad/file.shp'
pids = s2.get_product_ids('2021-11-01', '2021-12-31', 10, 5, 
                       shape_file=shape_file, bbox=None)

ERROR 5: cannot load source clip geometry


AttributeError: 'NoneType' object has no attribute 'GetLayer'

In [ ]:
lst = [(key, val, shape_file) for key, val in pids.items()]

In [ ]:
from multiprocessing import Pool, cpu_count, Process
pool = Pool(cpu_count()-2)
with pool:
    pool.map(ss.indices_generator, lst)

In [ ]:
# for key, val in pids.items():
    f_path = os.path.join('data', 'indice', str(key))
    try:
        os.makedirs(f_path)
    except OSError as exc:
        if exc.errno != errno.EEXIST:
            raise
        pass
    raster_list = defaultdict(list)
    for item in val:
        raster_list['b4_path'].append(s2.pid_to_path(item, 'B04'))
        raster_list['b8_path'].append(s2.pid_to_path(item, 'B08'))
        raster_list['b8a_path'].append(s2.pid_to_path(item, 'B8A'))
        raster_list['b11_path'].append(s2.pid_to_path(item, 'B11'))
    
    b4_file = merge_clip_raster(raster_file_list=raster_list['b4_path'], output_file='b4.tif', shp_file=shape_file)
    b8_file = merge_clip_raster(raster_file_list=raster_list['b8_path'], output_file='b8.tif', shp_file=shape_file)
    
    b4 = raster_to_array(b4_file, 'int16')
    b8 = raster_to_array(b8_file, 'int16')
    ndvi = get_ndvi(b4,b8)
    ndvi = np.around(ndvi, decimals=2, out=None)
    write_raster('b4.tif', ndvi, f'{f_path}/ndvi.tif', gdal.GDT_Float32)
    
    savi = get_savi(b4, b8, 0.428)
    savi = np.around(savi, decimals=2, out=None)
    write_raster('b4.tif', savi, f'{f_path}/savi.tif', gdal.GDT_Float32)
    b4 = b8 = ndvi = savi = None
    
    b8a_file = merge_clip_raster(raster_file_list=raster_list['b8a_path'], output_file='b8a.tif', shp_file=shape_file)
    b11_file = merge_clip_raster(raster_file_list=raster_list['b11_path'], output_file='b11.tif', shp_file=shape_file)
    b8a = raster_to_array(b8a_file, 'int16')
    b11 = raster_to_array(b11_file, 'int16')
    
    lswi = get_lswi(b8a, b11)
    lswi = np.around(lswi, decimals=2, out=None)
    write_raster('b8a.tif', lswi, f'{f_path}/lswi.tif', gdal.GDT_Float32)
    b8a = b11 = lswi = None